# DataWig Examples

## Installation

Clone the repository from git and set up virtualenv in the root dir of the package:

```
python3 -m venv venv
```

Install the package from local sources:

```
./venv/bin/pip install -e .
```

## Running DataWig
The DataWig API expects your data as a [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). Here is an example of how the dataframe might look:

|Product Type | Description           | Size | Color |
|-------------|-----------------------|------|-------|
|   Shoe      | Ideal for Running     | 12UK | Black |
| SDCards     | Best SDCard ever ...  | 8GB  | Blue  |
| Dress       | This **yellow** dress | M    | **?** |

DataWig let's you impute missing values in two ways:
  * A `.complete` functionality inspired by [`fancyimpute`](https://github.com/iskandr/fancyimpute)
  * A `sklearn`-like API with `.fit` and `.predict` methods

## Quickstart Example

### Using `AutoGluonImputer.complete`


In [1]:
# This allows to import datawig
from pathlib import Path
import sys,os
path_root = Path(os.getcwd()).parents[2]
sys.path.append(str(path_root))

In [2]:
import os, random, warnings
import numpy as np
import datawig

random.seed(0)
warnings.filterwarnings("ignore")

# generate some data with simple nonlinear dependency
df = datawig.utils.generate_df_numeric() 
# mask 10% of the values
df_with_missing = df.mask(np.random.rand(*df.shape) > .8)

# impute missing values
df_with_missing_imputed = datawig.AutoGluonImputer.complete(df_with_missing)

df['f(x) with_missing'] = df_with_missing['f(x)']
df['f(x) imputed'] = df_with_missing_imputed['f(x)']
df[-5:]

		No module named 'mxnet'
		No module named 'mxnet'


,x,f(x),f(x) with_missing,f(x) imputed
95,-0.038983,-0.006638,-0.006638,-0.006638
96,0.142835,0.019631,0.019631,0.019631
97,-0.455273,0.210685,0.210685,0.210685
98,-2.981880,8.894373,NaN,8.454887
99,-2.463691,6.078044,6.078044,6.078044


### Using `AutoGluonImputer.fit` and `.predict`

You can also impute values in specific columns only (called `output_column` below) using values in other columns (called `input_columns` below). DataWig currently supports imputation of categorical columns and numeric columns. Type inference is based on ``pandas`` 

#### Imputation of categorical columns

Let's first generate some random strings hidden in longer random strings:

In [3]:
df['f(x) with_missing'] = df_with_missing['f(x)']
df['f(x) imputed'] = df_with_missing_imputed['f(x)']

In [4]:
df = datawig.utils.generate_df_string( num_samples=200, 
                                       data_column_name='sentences', 
                                       label_column_name='label')
df.head(n=2)

,sentences,label
0,cMbm9 j7c1f RebhO BctvV m6Kop NQqEe,m6Kop
1,7zOfb O2NiT RwL85 Rz1TH G7Fgt m6Kop,m6Kop


In [5]:
df_train, df_test = datawig.utils.random_split(df)

imputer = datawig.AutoGluonImputer(
    input_columns=['sentences'], # column(s) containing information about the column we want to impute
    output_column='label' # the column we'd like to impute values for
    )

#Fit an imputer model on the train data
imputer.fit(train_df=df_train, time_limit=100)

#Impute missing values and return original dataframe with predictions
imputed = imputer.predict(df_test)
imputed.head(n=5)

		Unable to import dependency mxnet. A quick tip is to install via `pip install mxnet --upgrade`, or `pip install mxnet_cu101 --upgrade`


,sentences,label,label_imputed
57,PFTP5 IripW Wa0RH lm0lc Z9jZI DOBx3,Z9jZI,Z9jZI
31,O67IM lm0lc DdZ04 RwL85 n5RL0 Z9jZI,Z9jZI,Z9jZI
65,CH4F6 2V9SM Cffu4 Z9jZI zfx1h Rn9Xd,Z9jZI,Z9jZI
140,vvYxT lm0lc Z9jZI wy1Qq NQqEe OCyT4,Z9jZI,Z9jZI
89,ERA5K YkvB0 IlnyL Svkpo Z9jZI RwL85,Z9jZI,Z9jZI


#### Imputation of numerical columns

Imputation of numerical values works just like for categorical values.

Let's first generate some numeric values with a quadratic dependency:


In [6]:
import datawig

df = datawig.utils.generate_df_numeric( num_samples=200, 
                                        data_column_name='x', 
                                        label_column_name='y')         
df.head(n=5)

,x,y
0,1.895813,3.617395
1,-1.008764,1.024857
2,1.978105,3.919697
3,-2.638216,6.965940
4,2.480706,6.151376


In [7]:
df_train, df_test = datawig.utils.random_split(df)

imputer = datawig.AutoGluonImputer(
    input_columns=['x'], # column(s) containing information about the column we want to impute
    output_column='y', # the column we'd like to impute values for
    )

#Fit an imputer model on the train data
imputer.fit(train_df=df_train, time_limit=100)

#Impute missing values and return original dataframe with predictions
imputed = imputer.predict(df_test)
imputed.head(n=5)

		No module named 'mxnet'


,x,y,y_imputed
57,1.464692,2.149859,2.101123
31,-2.687957,7.225748,7.200390
65,2.226667,4.958026,5.079371
140,2.124441,4.502884,4.463196
89,-0.434246,0.176235,0.195624


In [8]:

import pandas as pd
from sklearn.metrics import classification_report

from sklearn.datasets import (
    load_diabetes,
    load_wine,
    make_hastie_10_2
)

def get_data(data_fn, noise=3e-1):
    X, y = data_fn(n_samples=10000)
    X = X + np.random.randn(*X.shape) * noise
    return pd.DataFrame(np.vstack([X.T, y]).T, columns= [str(i) for i in range(X.shape[-1] + 1)])




In [9]:

X = get_data(make_hastie_10_2)
label = X.columns[-1]
X[label] = X[label].astype(str)
features = X.columns[:-1]
df_train, df_test = datawig.utils.random_split(X.copy())

imputer = datawig.AutoGluonImputer(
    input_columns=[x for x in X.columns if x != label], # column(s) containing information about the column we want to impute
    output_column=label, # the column we'd like to impute values for
    verbosity=2,
    
)


In [10]:
#Fit an imputer model on the train data
imputer.fit(train_df=df_train, time_limit=10)


Beginning AutoGluon training ... Time limit = 10s
AutoGluon will save models to "./agModels/AutoGluonImputer/"
AutoGluon Version:  0.3.1
Train Data Rows:    7200
Train Data Columns: 10
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Train Data Class Count: 2
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    5870.25 MB
	Train Data (Original)  Memory Usage: 0.58 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 10 | ['0', '1', '2', '3', '4', ...]

In [18]:
features = X.columns[:-1]
precisions = []
for precision_threshold in [0.1, 0.5, 0.9, 0.95, .99]:
    imputed = imputer.predict(df_test[features], 
                          precision_threshold=precision_threshold, 
                          inplace=True)
    report = classification_report(df_test[label],imputed[label+"_imputed"].fillna(""), output_dict=True)
    precisions.append({
        'precision_threshold': precision_threshold,
        'empirical_precision_on_test_set': np.mean([report['-1.0']['precision'],report['1.0']['precision']])
    })

<AxesSubplot:>

In [22]:
pd.DataFrame(precisions).plot.bar()

In [26]:
df_precisions = pd.DataFrame(precisions)
df_precisions

,precision_threshold,empirical_precision_on_test_set
0,0.10,0.861234
1,0.50,0.861234
2,0.90,0.917931
3,0.95,0.962769
4,0.99,0.988919


In [33]:
df_precisions = pd.DataFrame(precisions)
precision_deviations = df_precisions['empirical_precision_on_test_set'] \
                            - df_precisions['precision_threshold'] + 0.01 
    
all(precision_deviations > 0)

True

In [34]:
from pathlib import Path
import sys,os
path_root = Path(os.getcwd()).parents[2]

In [35]:
path_root

Path('/Users')

In [36]:
os.getcwd()

'/Users/felix/code/datawig'